## This notebook explains some numbers in Section 2.1 of the paper. 

- Start from 2,199,705 ORCID iDs having both education and employment. Please refer to the notebook "0_data_extraction.ipynb" to review how these iDs are retrieved from the raw ORCID's 2022 public data.
- Among 2,199,705 ORCID iDs, 1,671,424 ORCID iDs have complete information about the country, institution, and title of all positions.
- Among 1,671,424 ORCID iDs, 172,169 ORCID iDs have both doctoral degrees and postdoctoral experiences. 
- Among 172,169 ORCID iDs, 125,364 ORCID iDs satisfied two conditions:
    - Complete information about the country, institution, title, and end date (year and month) of the doctoral degree
    - Complete information about the country, institution, title, and start date (year and month) of all jobs after the doctoral degree

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

### Load the cleaned data

In [2]:
df = pd.read_csv('../dat/orcid_2022_cleaned_df.csv', usecols = [0,1,2,3,4,5,6,8,9,10,11,13,14,16])
df = df.drop_duplicates()

df.columns = ['orcid', 'firstname', 'lastname', 'inst', 'city', 'region', 'country', 'department', 
              'title', 'start_year', 'start_month', 'end_year', 'end_month', 'source']
print(df.shape, len(set(df.orcid)))

(8467802, 14) 2199705


### Remove orcids with null countries and institutes if any

In [3]:
df = df[~df.orcid.isin(df[pd.isna(df.country)].orcid)]
print(df.shape, len(set(df.orcid)))

df = df[~df.orcid.isin(df[pd.isna(df.inst)].orcid)]
print(df.shape, len(set(df.orcid)))

(8465276, 14) 2199191
(8464965, 14) 2199129


### Remove orcids with nan title if any

In [4]:
df.title = df.title.apply(lambda x: str(x).lower().replace('.', ''))
df = df[~df.orcid.isin(df[(df.title=='nan') | (pd.isna(df.title))].orcid)]
print(df.shape, len(set(df.orcid)))

(6653687, 14) 1671424


### select ORCID iDs having both a doctoral degree and postdoctoral experience

In [5]:
doctor_condition = [len(x)!=0 for x in df.title.str.findall(r'ph(\s*)d|doctor of|dphil')]
df = df.assign(doctor = doctor_condition)
print(len(set(df[df.doctor==1].orcid)))

826959


In [6]:
postdoc_condition = [len(x)!=0 for x in df.title.str.findall(r'post[\-\s*]?doc')]
df = df.assign(postdoc = postdoc_condition)

In [7]:
target_orcids = set(df[df.doctor].orcid) & set(df[df.postdoc].orcid)
print(len(target_orcids))

172169


In [8]:
df = df[df.orcid.isin(target_orcids)].reset_index(drop=True)
print(df.shape, len(set(df.orcid)))

(944978, 16) 172169


### End year and month of a doctoral degree and all start years and months of the jobs after the doctoral degree should be provided

In [9]:
def concat_date(y, m):
    try:
        res = str(int(float(y)))+'/'+str(int(float(m))).zfill(2)
        return datetime.strptime(res, "%Y/%m")
    except:
        return None

In [10]:
doctor = df[df.doctor]
doctor = doctor.assign(end_date = [concat_date(y, m) 
                                     for y, m in zip(doctor.end_year, doctor.end_month)])
doctor = doctor[~pd.isna(doctor.end_date)]
last_doctor = doctor.sort_values(by=['end_date'], ascending=False).groupby('orcid').head(1).reset_index(drop=True)
print(last_doctor.shape)

(138425, 17)


In [11]:
non_doctor = df[~df.doctor]
non_doctor = non_doctor[non_doctor.orcid.isin(last_doctor.orcid)]
non_doctor = non_doctor.assign(start_date = [concat_date(y, m) 
                                             for y, m in zip(non_doctor.start_year, non_doctor.start_month)])
non_doctor = non_doctor.assign(end_date = [concat_date(y, m) 
                                             for y, m in zip(non_doctor.end_year, non_doctor.end_month)])

non_doctor = non_doctor[~pd.isna(non_doctor.start_date)].reset_index(drop=True)
print(non_doctor.shape, non_doctor.orcid.nunique())

non_doctor = non_doctor.merge(last_doctor[['orcid', 'end_date']], on='orcid')
non_doctor = non_doctor[non_doctor.start_date>=non_doctor.end_date_y]

print(non_doctor.shape, non_doctor.orcid.nunique())

(563956, 18) 137215
(335462, 19) 132645


In [12]:
postdoc_orcids = set(non_doctor[non_doctor.postdoc].orcid)
print(len(postdoc_orcids))

125364


In [13]:
last_doctor_for_postdoc = last_doctor[last_doctor.orcid.isin(postdoc_orcids)]
print(last_doctor_for_postdoc.shape)

non_doctor_for_postdoc = non_doctor[non_doctor.orcid.isin(postdoc_orcids)]
print(non_doctor_for_postdoc.shape)

(125364, 17)
(320847, 19)


In [14]:
df_ordered = pd.concat([last_doctor_for_postdoc, non_doctor_for_postdoc.iloc[:,:-2]], axis=0)
df_ordered = df_ordered.sort_values(by=['orcid', 'doctor', 'start_date'], ascending=[True, False, True])
df_ordered = df_ordered.drop(['city', 'region', 'source'], axis=1)

df_ordered = df_ordered.drop_duplicates().reset_index(drop=True)

print(df_ordered.shape, df_ordered.orcid.nunique())

(445869, 15) 125364


In [15]:
df_ordered.to_csv('_dat_orcid_2022_postdoc_trajectories.csv.gz', index=False, compression='gzip')